<a href="https://colab.research.google.com/github/STKKKKK/UTS_41004_ID12431868/blob/master/acp_a2_Group_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 41004_A2
# Project 9-Topic 4: Multi-Objective Optimization

Github Link: 

**members of the group#30:**<br>
Yi Lin      12444129 <br>
Shiwei Xu   12418986 <br>
Yuxiang Xu  12715948 <br>
Hong Kung   12431868 <br>

##  Problem description  

Our data analytics company will help our customer deal with the MOO(multi-objective optimization) <br>problems in their database. As it is shown later, The dataset is about social network analysis,<br> include 1000 topic nodes and their 5 rankings for each attribute.<br>
Our current goal is to re-rank these nodes by multiple attributes. Three traditional approaches <br>will be implement to solve this MOO problem: scalarization, machine learning, non-dominated sorting<br>(this is what we are mainly focused on and will be developed in future tasks)  <br>

## Data exploration and initial findings

In this section, we will prepare the dataset we use and show what we have dicovered so far.

In [0]:
#Importing required modules
import numpy as np
import pandas as pd
import math
import random

In [54]:
# The excel dataset have been uploaded to github
url = 'https://raw.githubusercontent.com/STKKKKK/UTS_41004_ID12431868/master/Topic4_IS_multipleRanking.xlsx'
df = pd.read_excel(url)

df.head()

,Node,Degree Centrality,Closeness Centrality,Between Centrality,Connectivity,Externality
0,Bibliometrics,3905.924925,0.806295,0.021293,9.620818,62045
1,Information retrieval,2157.933934,0.753394,0.018668,6.956349,26450
2,collaboration,2465.552553,0.793487,0.021247,6.073034,28579
3,United States,2077.891892,0.770833,0.018114,5.764045,23063
4,case study,1771.111111,0.794118,0.021508,2.436242,21383


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Node                  1000 non-null   object 
 1   Degree Centrality     1000 non-null   float64
 2   Closeness Centrality  1000 non-null   float64
 3   Between Centrality    1000 non-null   float64
 4   Connectivity          1000 non-null   float64
 5   Externality           1000 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 47.0+ KB


The dataset have 1000 rows and 5 columns. And fortunately, there is no null value.

Apart from **df**, we can also create **df2, df3** which have a different object type, <br>since this is convenient for different usages.

In [0]:
# df is the original pandas dataframe
df2 = np.array(df)      # df2 is a 2-dimensional numpy array
df3 = df2.T             # df3 convert the x and y-axis of df2

In [57]:
print(df2.shape)
print(df3.shape)

(1000, 6)
(6, 1000)


## Difficulties encountered & Updated project plan 

The main difficulty of MOO is to build the model.In this section, we will process the data with <br>three traditional approaches: scalarization, non-dominated sorting and machine learning. And we <br>will also update project plan under each model we build.

### Scalarization

Scalarization is a basic method to deal with MOO. It is relatively simple in classical technology. <br>Its main step is to combine multiple objective functions into a whole objective function, and <br>determine the weight before the optimization process.<br>
The optimal solution is controlled by the weight vector. In our case, we will use the equal-weights <br>as an easy example.

In [0]:
def equal_weights(*args):  
    sums = np.array([0 for i in range(1000)])
    n = len(args)
    for x in args:
        df_sorted = df.sort_values(by = x)
        df_np = (np.array(df_sorted)).T

        rank = 1
        for node in df_np[0]:
            i = list(df['Node']).index(node)
            sums[i] = sums[i] + rank
            rank = rank + 1
    return 1000 - sums/n

In [59]:
# Then we test the function with 3 attributes.
res = equal_weights('Closeness Centrality', 'Connectivity', 'Externality')
# We print out the rank of first 10 nodes to see if works.
print(res[:10])
print('\n', 'There are', len(res), 'ranks in 1000 nodes!')

[ 0.          3.          1.66666667  3.66666667 11.66666667 30.
 12.66666667  3.66666667 14.         64.66666667]

 There are 1000 ranks in 1000 nodes!


**Updated project plan:**<br>
Although we can get a very long ranking list which is one of the main goal of MOO problem. <br>However in reality, a solution with equal weight for all goals may be almost nonexistent. It <br>is good for us to look for a more robust model to solve with the MOO problem, Non-Dominated Sorting <br>and NSGA(Non-Dominated Sorting Genetic Algorithm) will be implemented in the following task.


### Non-Dominated Sorting

Non-dominated sorting is the most widely used selection operation of MOO problems. <br>
Initially, we need to define what is: point A(m1,m2,m3...) 'dominate' another point B(n1,n2,n3...).

In [0]:
def dominate(cols, a, b):   
    """
    return 1: true
    return 0: false
    return -1: neither OR 'a,b are the same point'
    """    
    n = len(cols)
    flags = []
    for col in cols:
        if col[a] > col[b]:
            flags.append('greater')
        elif col[a] < col[b]:
            flags.append('smaller')
        elif col[a] == col[b]:
            flags.append('equal')
    
    if ('greater' not in flags and 'smaller' not in flags):
        return -1
    elif ('smaller' not in flags):
        return 1
    elif ('greater' not in flags):
        return 0
    else:
        return -1

Non-dominated sorting means to sort the solutions in population according to <br>the **Pareto dominance principle.**

In [0]:
def fast_non_dominated_sort(cols):
    """
    This fuction will return the Pareto front after sorting.
    Front: Pareto front
    Dom: the dominate list of each point.
    n: the 'inferior level' of each point.
        -- for example: n[A]==0 :point A is not dominated by any point.
                        n[A]==2 :point A is dominated by all points with n[B]==1,
                                 and dominates all points with n[C]==3.
    rank: rank of the point.
    """
    Front = [[]]
    Dom = [[] for i in range(0,len(cols[0]))]
    n = [0 for i in range(0,len(cols[0]))]

    for f in range(0,len(cols[0])):
        for g in range(0, len(cols[0])):
            if (dominate(cols, f, g) == 1) and not(g in Dom[f]):
                Dom[f].append(g)
            elif (dominate(cols, f, g) == 0):
                n[f] = n[f] +1        
        if (n[f] == 0) and not(f in Front[0]):
            Front[0].append(f)

    rank = 0
    while(Front[rank] != []):
        Q=[]
        for f in Front[rank]:
            for g in Dom[f]:
                n[g] = n[g] -1
                if (n[g] == 0) and not(g in Q):
                     Q.append(g)
        rank = rank +1
        Front.append(Q)

    Front.pop()
    return Front

At the first stage, we defined the non-dominated sort function which can only carry out 2 columns<br>(attribute) for each node. By reviewing several concepts of Pareto dominance, we finally decide <br>to define a 'dominate' function individually, from that we can easily handle out any numbers of attributes. 


In [62]:
# Then we test the function with 3 attributes
res2 = fast_non_dominated_sort(df3[1:4])
# we print out first 7 ranks to see which node(node index) is included  
print(res2[:7])
print('\n', 'There are', len(res2), 'ranks in 1000 nodes!')

[[0, 4], [2], [1, 3], [5, 7], [11, 13, 18], [29, 12, 6, 21], [10, 16]]

 There are 114 ranks in 1000 nodes!


In [46]:
# We can also test the function with all attributes
res3 = fast_non_dominated_sort(df3[:])
# we print out first 7 ranks to see which node(node index) is included  
print(*res3[:7], sep='\n')
print('\n', 'There are', len(res3), 'ranks in 1000 nodes!')

[0, 1, 2, 4, 5, 6, 12, 13, 18, 19, 21, 29, 32, 40, 46, 55, 90, 112, 134, 213, 338, 849, 986]
[3, 7, 16, 34, 43, 22, 24, 14, 9, 33, 45, 17, 20, 25, 26, 41, 42, 44, 92, 106, 69, 143, 158, 179, 212, 229, 159, 194, 204, 241, 262, 329, 373, 451, 520, 521, 337, 485]
[10, 11, 15, 8, 28, 35, 39, 60, 27, 63, 70, 23, 31, 37, 38, 67, 49, 52, 56, 62, 73, 79, 89, 172, 310, 311, 328, 284, 308, 291, 243, 293, 233, 275, 372, 312, 319, 418, 467, 591, 522, 797, 847, 888, 931]
[81, 66, 86, 88, 110, 127, 239, 53, 58, 59, 30, 36, 57, 64, 74, 75, 78, 99, 121, 50, 65, 84, 87, 91, 126, 107, 131, 417, 449, 466, 547, 565, 383, 242, 540, 796, 464, 366, 261, 401, 431, 450, 606, 721, 336, 365, 542, 543, 628, 984]
[346, 125, 138, 274, 149, 164, 178, 183, 217, 232, 246, 249, 171, 83, 47, 103, 123, 162, 72, 76, 80, 85, 93, 94, 96, 104, 111, 122, 147, 98, 109, 114, 130, 145, 146, 148, 231, 182, 200, 48, 61, 77, 101, 165, 586, 719, 500, 627, 654, 714, 792, 839, 885, 564, 626, 712, 484, 545, 318, 400, 516, 546, 607, 757

**Updated project plan:**<br>
According to the ranking results, it is shown that the more objects(attributs) we need to handle out, <br>the shorter length we have for the ranking list, and more point will convergence to the same rank <br>(before they are confirmed as 'very accurate'). <br>
These issues are not acceptable in MOO, and must be solved in the future task. Since we definitely <br>can't decide how many objects(attribute) we need to handle in the bussiness problem, we have <br>to operate more with the ranking result we have now. Genetic Algorithm will be used, and the <br>first step is to calculate the crowd distance of different points. Many operation also need to <br>be built, then we will finally build a robust NSGA.<br>
Moreover, In our opinion, machine leaning methods can be also used in MOO. In current assignment, we <br>will just build the model, and use the non-dominated sorting result as the train dataset.

### Machine Learning

In [0]:
# Importing required modules for Machine learning
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics #for checking the model accuracy

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

The result from Non-Dominated Sort will be used in this section. Note that we will use the res2, which only handle out the object(attribute) No.2,3,4.

In [67]:
df['rank'] = 0      #should create the column 'rank' first

rank = 0
for m in res2:
    for node_id in m:
        df['rank'][node_id] = rank
    rank = rank +1

df['range'] = df['rank'].apply(
    lambda x: (x < 11 and '0~10') or (x < 21 and '11~20') or (x < 31 and '21~30') 
         or (x < 41 and '31~40') or (x < 51 and '41~50') or (x < 61 and '51~60')
         or (x < 71 and '61~70') or (x < 81 and '71~80') or (x < 91 and '81~90')
         or (x < 101 and '91~100') or '101~114')

# Look we have now
df.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Node,Degree Centrality,Closeness Centrality,Between Centrality,Connectivity,Externality,rank,range
995,co-citation networks,11.009009,0.524685,0.000130,0.331461,873,77,71~80
996,Cognitive Science,7.185185,0.519231,0.000101,0.164430,664,92,91~100
997,cognitive structure,7.986987,0.520584,0.000136,0.230337,2126,86,81~90
998,Congress Subject Headings,6.964965,0.518422,0.000117,0.123016,308,90,81~90
999,cost-effectiveness,4.624625,0.517081,0.000121,0.063492,378,92,91~100


We want 700 nodes used for training, and the rest of the nodes used for testing

In [70]:
train, test = train_test_split(df, test_size = 0.3)
print(train.shape)
print(test.shape)

(700, 8)
(300, 8)


In [0]:
Set = ['Closeness Centrality','Between Centrality','Connectivity']

train_X = train[Set]
train_y = train['range']	
test_X = test[Set] 
test_y = test['range']

By the first, we build a ID3 model.

In [82]:
model1 = DecisionTreeClassifier(criterion="entropy")
model1.fit(train_X, train_y)
prediction = model1.predict(test_X)
print('The accuracy of the Decision Tree (ID3) is',metrics.accuracy_score(prediction,test_y))

The accuracy of the Decision Tree (ID3) is 0.85


The accuracy is satisfy. We can also build some ensemble models to see whether they will have better performance.

In [73]:
model2 = BaggingClassifier(base_estimator=model1, n_estimators=10, max_samples=0.8, max_features=0.8)
model2.fit(train_X, train_y)
prediction = model2.predict(test_X)
print('The accuracy of the Bagging is',metrics.accuracy_score(prediction,test_y))

The accuracy of the Bagging is 0.8833333333333333


In [85]:
model3 = AdaBoostClassifier(base_estimator=model1, n_estimators=10)
model3.fit(train_X, train_y)
prediction = model3.predict(test_X)
print('The accuracy of the AdaBoosting is',metrics.accuracy_score(prediction,test_y))

The accuracy of the AdaBoosting is 0.8533333333333334


According to the result which runs several times, we find that bagging method have higher accuracy <br>than individual ID3 model, while AdaBoosting does not significantly perform better.

**Updated project plan:**<br>

Since we haven't yet develop the non-dominated sort to deal with the ranking result in deeper <br>aspect. We will keep the machine learning model to be used in future task. No matter if <br>machine learning really have its effect to deal with the problems in our non-dominated sort <br>result. We can still keep the idea when we develop our genetic algorithm model in the future.